In [1]:
import numpy as np
import pandas as pd
from sergio import sergio

import sys
locate_python = sys.exec_prefix
print(locate_python)

C:\Users\moha456\Anaconda3\envs\my_env


In [2]:
print("My numpy version is: ", np.__version__)

My numpy version is:  1.23.1


# Run SERGIO

In [3]:
sim = sergio(number_genes=100, number_bins = 9, number_sc = 300, noise_params = 1, decays=0.8, sampling_state=15, noise_type='dpd')
sim.build_graph(input_file_taregts ='steady-state_input_GRN.txt', input_file_regs='steady-state_input_MRs.txt', shared_coop_state=2)
sim.simulate()
expr = sim.getExpressions()
expr_clean_ss = np.concatenate(expr, axis = 1)
expr_clean_ss[1:5]

Start simulating new level
There are 4 genes to simulate in this layer
Done with current level
Start simulating new level
There are 6 genes to simulate in this layer
Done with current level
Start simulating new level
There are 90 genes to simulate in this layer
Done with current level


array([[ 4.17270032,  7.06277116, 12.04672079, ...,  2.43186517,
         3.80522853,  5.17320451],
       [ 0.65084038,  1.68489675,  1.6310032 , ...,  1.30904445,
         0.79825537,  2.80956254],
       [ 1.13787638,  0.08549465,  0.99698709, ...,  1.16441128,
         1.53895431,  2.07365252],
       [ 3.72821276,  5.88036121,  1.51093267, ...,  1.76231963,
         2.60224582,  1.98744161]])

In [4]:
expr_clean_ss 
df_output = pd.DataFrame(expr_clean_ss) 
df_output 

,0,1,2,3,4,5,6,7,8,9,...,2690,2691,2692,2693,2694,2695,2696,2697,2698,2699
0,5.742781,3.681879,5.949217,6.634987,4.078987,5.032606,5.105771,2.041763,1.916706,5.485587,...,0.640721,2.459189,0.742016,0.636186,1.868894,4.242222,2.772861,3.255742,2.162107,2.436828
1,4.172700,7.062771,12.046721,8.049915,6.103824,12.470252,5.494164,9.169878,5.577372,7.188532,...,4.313052,2.748536,2.907056,1.020130,1.266793,3.407464,5.191953,2.431865,3.805229,5.173205
2,0.650840,1.684897,1.631003,3.281255,2.826805,2.459554,0.247129,2.370296,2.623522,2.652369,...,1.268460,0.000000,0.988558,1.333052,0.388661,0.827836,2.492165,1.309044,0.798255,2.809563
3,1.137876,0.085495,0.996987,1.286907,0.615787,0.719535,0.371324,0.076793,1.791152,2.904399,...,1.351451,0.216266,2.381308,0.011623,1.475167,1.348169,0.980611,1.164411,1.538954,2.073653
4,3.728213,5.880361,1.510933,5.843811,2.578874,0.786477,2.851539,4.953962,1.196158,9.953144,...,0.280569,0.126349,1.508773,0.415624,0.904260,1.695689,2.682445,1.762320,2.602246,1.987442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.742263,7.984740,1.361126,1.229594,0.683944,1.578191,4.002056,6.328576,0.870237,5.570494,...,0.818151,2.428066,0.277011,4.652609,4.370747,1.187071,1.602446,4.954746,3.681582,2.125434
96,5.757358,3.305608,5.100027,4.273794,4.611466,2.270888,4.174557,3.456165,4.712684,3.910054,...,3.212612,1.098460,2.530560,1.560354,1.651840,0.869588,3.874374,1.504029,0.245044,3.032343
97,4.937538,7.941098,5.703930,4.618157,6.055777,4.816284,4.180223,7.759709,6.312207,4.391117,...,2.749406,7.343569,4.096416,6.802611,3.598192,2.717292,5.662001,3.442603,2.046301,6.406387
98,4.946284,6.934141,14.460026,6.215145,12.627603,14.948989,4.806217,7.821267,15.117584,8.782147,...,9.700578,9.689161,9.326278,14.256531,12.203686,12.372031,9.331101,8.691783,6.987876,8.948801


In [5]:
# save to csv
df_output.to_csv('GeneExpression_no_perturb.csv') 

# Add technical noise

In [6]:
"""
Add outlier genes
"""
expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = 82)
expr_O_L_D = np.multiply(binary_ind, expr_O_L)

"""
Convert to UMI count
"""
count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)

"""
Make a 2d gene expression matrix
"""
count_matrix = np.concatenate(count_matrix, axis = 1)

In [7]:
count_matrix = sim.convert_to_UMIcounts(expr_O)
expr_tech_ss_O = np.concatenate(count_matrix, axis = 1)
expr_tech_ss_O[1:5]

array([[ 5,  3, 20, ...,  1,  1, 10],
       [ 1,  3,  4, ...,  0,  0,  2],
       [ 2,  0,  1, ...,  1,  1,  1],
       [ 2,  6,  0, ...,  0,  3,  1]])

In [8]:
count_matrix = sim.convert_to_UMIcounts(expr_O_L)
expr_tech_ss_O_L = np.concatenate(count_matrix, axis = 1)
expr_tech_ss_O_L[1:5]

array([[1, 1, 2, ..., 2, 0, 1],
       [2, 0, 0, ..., 2, 0, 3],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 2, 0, ..., 0, 2, 0]])

In [9]:
count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)
expr_tech_ss_O_L_D = np.concatenate(count_matrix, axis = 1)
expr_tech_ss_O_L_D[1:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [10]:
df_output = pd.DataFrame(expr_tech_ss_O) 
df_output.to_csv('GeneExpression_no_perturb_tech_O.csv') 

In [11]:
df_output = pd.DataFrame(expr_tech_ss_O_L_D) 
df_output.to_csv('GeneExpression_no_perturb_tech_O_L_D.csv') 

In [12]:
df_output = pd.DataFrame(expr_tech_ss_O_L) 
df_output.to_csv('GeneExpression_no_perturb_tech_O_L.csv') 